## Bygg en logistisk regresjonsmodell - Leksjon 4

![Infografikk om logistisk vs. lineær regresjon](../../../../../../2-Regression/4-Logistic/images/linear-vs-logistic.png)

#### **[Quiz før leksjon](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

#### Introduksjon

I denne siste leksjonen om regresjon, en av de grunnleggende *klassiske* ML-teknikkene, skal vi se nærmere på logistisk regresjon. Du kan bruke denne teknikken for å oppdage mønstre og forutsi binære kategorier. Er dette godteri sjokolade eller ikke? Er denne sykdommen smittsom eller ikke? Vil denne kunden velge dette produktet eller ikke?

I denne leksjonen vil du lære:

-   Teknikker for logistisk regresjon

✅ Fordyp deg i hvordan du arbeider med denne typen regresjon i dette [Learn-modulet](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott)

## Forutsetning

Etter å ha jobbet med gresskar-dataene, er vi nå kjent nok med dem til å innse at det finnes én binær kategori vi kan jobbe med: `Color`.

La oss bygge en logistisk regresjonsmodell for å forutsi, gitt noen variabler, *hvilken farge et gitt gresskar sannsynligvis har* (oransje 🎃 eller hvit 👻).

> Hvorfor snakker vi om binær klassifisering i en leksjonsgruppe om regresjon? Kun av språklig bekvemmelighet, ettersom logistisk regresjon [egentlig er en klassifiseringsmetode](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression), om enn en lineær-basert en. Lær om andre måter å klassifisere data på i neste leksjonsgruppe.

For denne leksjonen trenger vi følgende pakker:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) er en [samling av R-pakker](https://www.tidyverse.org/packages) designet for å gjøre datavitenskap raskere, enklere og morsommere!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) er et [rammeverk av pakker](https://www.tidymodels.org/packages/) for modellering og maskinlæring.

-   `janitor`: [janitor-pakken](https://github.com/sfirke/janitor) gir enkle verktøy for å undersøke og rense rotete data.

-   `ggbeeswarm`: [ggbeeswarm-pakken](https://github.com/eclarke/ggbeeswarm) gir metoder for å lage beeswarm-stil grafer ved hjelp av ggplot2.

Du kan installere dem slik:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

Alternativt kan skriptet nedenfor sjekke om du har de nødvendige pakkene for å fullføre dette modulen og installere dem for deg hvis de mangler.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **Definer spørsmålet**

For vårt formål vil vi uttrykke dette som en binær: 'Hvit' eller 'Ikke Hvit'. Det finnes også en kategori for 'stripete' i datasettet vårt, men det er få forekomster av den, så vi kommer ikke til å bruke den. Den forsvinner uansett når vi fjerner nullverdier fra datasettet.

> 🎃 Artig fakta: Vi kaller noen ganger hvite gresskar for 'spøkelsesgresskar'. De er ikke så lette å skjære ut, så de er ikke like populære som de oransje, men de ser kule ut! Så vi kunne også omformulert spørsmålet vårt til: 'Spøkelse' eller 'Ikke Spøkelse'. 👻

## **Om logistisk regresjon**

Logistisk regresjon skiller seg fra lineær regresjon, som du har lært om tidligere, på noen viktige måter.

#### **Binær klassifisering**

Logistisk regresjon tilbyr ikke de samme funksjonene som lineær regresjon. Førstnevnte gir en prediksjon om en `binær kategori` ("oransje eller ikke oransje"), mens sistnevnte er i stand til å forutsi `kontinuerlige verdier`, for eksempel gitt opprinnelsen til et gresskar og tidspunktet for innhøsting, *hvor mye prisen vil stige*.

![Infografikk av Dasani Madipalli](../../../../../../2-Regression/4-Logistic/images/pumpkin-classifier.png)

### Andre klassifiseringer

Det finnes andre typer logistisk regresjon, inkludert multinomial og ordinal:

- **Multinomial**, som innebærer å ha mer enn én kategori - "Oransje, Hvite og Stripete".

- **Ordinal**, som innebærer ordnede kategorier, nyttig hvis vi ønsker å ordne resultatene våre logisk, som gresskarene våre som er ordnet etter et begrenset antall størrelser (mini,sm,med,lg,xl,xxl).

![Multinomial vs ordinal regresjon](../../../../../../2-Regression/4-Logistic/images/multinomial-vs-ordinal.png)

#### **Variabler TRENGER IKKE å korrelere**

Husker du hvordan lineær regresjon fungerte bedre med mer korrelerte variabler? Logistisk regresjon er det motsatte - variablene trenger ikke å være i samsvar. Dette fungerer for disse dataene som har noe svake korrelasjoner.

#### **Du trenger mye rene data**

Logistisk regresjon gir mer nøyaktige resultater hvis du bruker mer data; vårt lille datasett er ikke optimalt for denne oppgaven, så husk det.

✅ Tenk på hvilke typer data som egner seg godt for logistisk regresjon

## Øvelse - rydd opp i dataene

Først, rydd opp i dataene litt, fjern nullverdier og velg kun noen av kolonnene:

1. Legg til følgende kode:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


Du kan alltid ta en titt på din nye dataframe ved å bruke [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html)-funksjonen som vist nedenfor:


In [ ]:
pumpkins_select %>% 
  glimpse()


La oss bekrefte at vi faktisk skal jobbe med et binært klassifiseringsproblem:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### Visualisering - kategorisk plot
Nå har du lastet opp gresskar-dataene igjen og renset dem slik at du har et datasett som inneholder noen få variabler, inkludert Farge. La oss visualisere dataframen i notatboken ved hjelp av ggplot-biblioteket.

Ggplot-biblioteket tilbyr noen smarte måter å visualisere dataene dine på. For eksempel kan du sammenligne distribusjoner av dataene for hver Sort og Farge i et kategorisk plot.

1. Lag et slikt plot ved å bruke geombar-funksjonen, med gresskar-dataene våre, og spesifiser en fargekartlegging for hver gresskarkategori (oransje eller hvit):


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

Ved å observere dataene kan du se hvordan Farge-dataene henger sammen med Variant.

✅ Gitt dette kategoriske diagrammet, hvilke interessante utforskninger kan du se for deg?


### Databehandling: funksjonskoding

Datasettet vårt med gresskar inneholder strengverdier for alle kolonnene. Å jobbe med kategoriske data er intuitivt for mennesker, men ikke for maskiner. Maskinlæringsalgoritmer fungerer godt med tall. Derfor er koding et veldig viktig steg i databehandlingsfasen, siden det lar oss gjøre kategoriske data om til numeriske data uten å miste informasjon. God koding fører til å bygge en god modell.

For funksjonskoding finnes det to hovedtyper av kodere:

1. Ordinal koder: denne passer godt for ordinale variabler, som er kategoriske variabler der dataene følger en logisk rekkefølge, som kolonnen `item_size` i datasettet vårt. Den lager en mapping slik at hver kategori representeres av et tall, som er rekkefølgen til kategorien i kolonnen.

2. Kategorisk koder: denne passer godt for nominelle variabler, som er kategoriske variabler der dataene ikke følger en logisk rekkefølge, som alle funksjonene som er forskjellige fra `item_size` i datasettet vårt. Det er en one-hot encoding, som betyr at hver kategori representeres av en binær kolonne: den kodede variabelen er lik 1 hvis gresskaret tilhører den sorten, og 0 ellers.

Tidymodels tilbyr enda en praktisk pakke: [recipes](https://recipes.tidymodels.org/) - en pakke for databehandling. Vi definerer en `recipe` som spesifiserer at alle prediktorkolonner skal kodes til et sett med heltall, `prep` den for å estimere de nødvendige mengdene og statistikkene som trengs for operasjonene, og til slutt `bake` for å bruke beregningene på nye data.

> Vanligvis brukes recipes som en forbehandler for modellering, der den definerer hvilke steg som skal brukes på et datasett for å gjøre det klart for modellering. I så fall er det **sterkt anbefalt** at du bruker en `workflow()` i stedet for å manuelt estimere en oppskrift med prep og bake. Vi skal se alt dette om et øyeblikk.
>
> Men for nå bruker vi recipes + prep + bake for å spesifisere hvilke steg som skal brukes på et datasett for å gjøre det klart for dataanalyse, og deretter hente ut de forhåndsbehandlede dataene med de anvendte stegene.


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ Hva er fordelene med å bruke en ordinal encoder for Item Size-kolonnen?

### Analyser forholdet mellom variabler

Nå som vi har forhåndsprosesseret dataene våre, kan vi analysere forholdet mellom funksjonene og etiketten for å få en idé om hvor godt modellen vil kunne forutsi etiketten basert på funksjonene. Den beste måten å utføre denne typen analyse på er å visualisere dataene.  
Vi skal igjen bruke ggplot geom_boxplot_-funksjonen for å visualisere forholdet mellom Item Size, Variety og Color i et kategorisk diagram. For å bedre visualisere dataene vil vi bruke den kodede Item Size-kolonnen og den ukodede Variety-kolonnen.


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### Bruk et swarm-plot

Siden Color er en binær kategori (Hvit eller Ikke), krever det 'en [spesialisert tilnærming](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf) til visualisering'.

Prøv et `swarm-plot` for å vise fordelingen av farge med hensyn til item_size.

Vi skal bruke [ggbeeswarm-pakken](https://github.com/eclarke/ggbeeswarm), som gir metoder for å lage beeswarm-lignende plott ved hjelp av ggplot2. Beeswarm-plott er en måte å plotte punkter som normalt ville overlappet, slik at de i stedet plasseres ved siden av hverandre.


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


Nå som vi har en idé om forholdet mellom de binære kategoriene for farge og den større gruppen av størrelser, la oss utforske logistisk regresjon for å bestemme den sannsynlige fargen på et gitt gresskar.

## Bygg modellen din

Velg variablene du vil bruke i klassifiseringsmodellen din, og del dataene inn i trenings- og testsett. [rsample](https://rsample.tidymodels.org/), en pakke i Tidymodels, gir infrastruktur for effektiv datadeling og resampling:


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 Vi er nå klare til å trene en modell ved å tilpasse treningsfunksjonene til treningsetiketten (farge).

Vi starter med å lage en oppskrift som spesifiserer forbehandlingsstegene som skal utføres på dataene våre for å gjøre dem klare for modellering, dvs. koding av kategoriske variabler til et sett med heltall. Akkurat som `baked_pumpkins`, lager vi en `pumpkins_recipe`, men vi bruker ikke `prep` og `bake`, siden dette vil bli inkludert i en arbeidsflyt, som du vil se om noen få steg.

Det finnes ganske mange måter å spesifisere en logistisk regresjonsmodell i Tidymodels. Se `?logistic_reg()`. For nå vil vi spesifisere en logistisk regresjonsmodell via standardmotoren `stats::glm()`.


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


Nå som vi har en oppskrift og en modellspecifikasjon, trenger vi en måte å samle dem sammen i et objekt som først vil forhåndsbehandle dataene (prep+bake i bakgrunnen), tilpasse modellen på de forhåndsbehandlede dataene og også tillate eventuelle etterbehandlingsaktiviteter.

I Tidymodels kalles dette praktiske objektet en [`workflow`](https://workflows.tidymodels.org/) og holder på en praktisk måte dine modelleringskomponenter.


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


Etter at en arbeidsflyt har blitt *spesifisert*, kan en modell `trenings` ved hjelp av [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html)-funksjonen. Arbeidsflyten vil estimere en oppskrift og forhåndsbehandle dataene før trening, så vi slipper å gjøre det manuelt ved hjelp av prep og bake.


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


Modellen viser koeffisientene som ble lært under treningen.

Nå som vi har trent modellen med treningsdataene, kan vi gjøre prediksjoner på testdataene ved hjelp av [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). La oss starte med å bruke modellen til å forutsi etiketter for testsettet og sannsynlighetene for hver etikett. Når sannsynligheten er større enn 0,5, er den predikerte klassen `WHITE`, ellers `ORANGE`.


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


Veldig bra! Dette gir litt mer innsikt i hvordan logistisk regresjon fungerer.

### Bedre forståelse gjennom en forvirringsmatrise

Å sammenligne hver prediksjon med dens tilsvarende "sanne verdi" er ikke en særlig effektiv måte å avgjøre hvor godt modellen predikerer. Heldigvis har Tidymodels noen flere triks i ermet: [`yardstick`](https://yardstick.tidymodels.org/) - en pakke som brukes til å måle modellers effektivitet ved hjelp av ytelsesmetrikker.

En ytelsesmetrikk som er knyttet til klassifiseringsproblemer er [`forvirringsmatrisen`](https://wikipedia.org/wiki/Confusion_matrix). En forvirringsmatrise beskriver hvor godt en klassifiseringsmodell presterer. Den viser hvor mange eksempler i hver klasse som ble korrekt klassifisert av en modell. I vårt tilfelle vil den vise hvor mange oransje gresskar som ble klassifisert som oransje, og hvor mange hvite gresskar som ble klassifisert som hvite; forvirringsmatrisen viser også hvor mange som ble klassifisert i **feil** kategorier.

Funksjonen [**`conf_mat()`**](https://tidymodels.github.io/yardstick/reference/conf_mat.html) fra yardstick beregner denne krysstabuleringen av observerte og predikerte klasser.


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


La oss tolke forvirringsmatrisen. Modellen vår blir bedt om å klassifisere gresskar i to binære kategorier, kategorien `hvit` og kategorien `ikke-hvit`.

-   Hvis modellen din forutsier et gresskar som hvitt og det faktisk tilhører kategorien 'hvit', kaller vi det en `sann positiv` (true positive), vist med tallet øverst til venstre.

-   Hvis modellen din forutsier et gresskar som ikke hvitt og det faktisk tilhører kategorien 'hvit', kaller vi det en `falsk negativ` (false negative), vist med tallet nederst til venstre.

-   Hvis modellen din forutsier et gresskar som hvitt og det faktisk tilhører kategorien 'ikke-hvit', kaller vi det en `falsk positiv` (false positive), vist med tallet øverst til høyre.

-   Hvis modellen din forutsier et gresskar som ikke hvitt og det faktisk tilhører kategorien 'ikke-hvit', kaller vi det en `sann negativ` (true negative), vist med tallet nederst til høyre.

| Sannhet |
|:-------:|

|               |        |       |
|---------------|--------|-------|
| **Forutsagt** | HVIT   | ORANSJE |
| HVIT          | TP     | FP    |
| ORANSJE       | FN     | TN    |

Som du kanskje har gjettet, er det å foretrekke å ha et høyere antall sanne positive og sanne negative, og et lavere antall falske positive og falske negative, noe som indikerer at modellen presterer bedre.

Forvirringsmatrisen er nyttig fordi den gir opphav til andre måleparametere som kan hjelpe oss med å evaluere ytelsen til en klassifiseringsmodell bedre. La oss gå gjennom noen av dem:

🎓 Presisjon: `TP/(TP + FP)` definert som andelen av forutsagte positive som faktisk er positive. Også kalt [positiv prediktiv verdi](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value").

🎓 Gjenkalling: `TP/(TP + FN)` definert som andelen positive resultater av antall prøver som faktisk var positive. Også kjent som `sensitivitet`.

🎓 Spesifisitet: `TN/(TN + FP)` definert som andelen negative resultater av antall prøver som faktisk var negative.

🎓 Nøyaktighet: `TP + TN/(TP + TN + FP + FN)` Prosentandelen av etiketter som er korrekt forutsagt for en prøve.

🎓 F-mål: Et vektet gjennomsnitt av presisjon og gjenkalling, der det beste er 1 og det dårligste er 0.

La oss beregne disse måleparametrene!


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## Visualiser ROC-kurven for denne modellen

La oss gjøre en visualisering til for å se den såkalte [`ROC-kurven`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic):


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


ROC-kurver brukes ofte for å få en oversikt over resultatene til en klassifiserer når det gjelder sanne vs. falske positive. ROC-kurver viser vanligvis `True Positive Rate`/Sensitivitet på Y-aksen og `False Positive Rate`/1-Spesifisitet på X-aksen. Derfor er kurvens bratthet og avstanden mellom midtlinjen og kurven viktig: du ønsker en kurve som raskt går opp og over linjen. I vårt tilfelle er det falske positive i starten, og deretter går linjen opp og over på riktig måte.

Til slutt, la oss bruke `yardstick::roc_auc()` for å beregne det faktiske arealet under kurven (AUC). En måte å tolke AUC på er som sannsynligheten for at modellen rangerer et tilfeldig positivt eksempel høyere enn et tilfeldig negativt eksempel.


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


Resultatet er rundt `0.975`. Gitt at AUC varierer fra 0 til 1, ønsker du en høy score, siden en modell som er 100 % korrekt i sine prediksjoner vil ha en AUC på 1; i dette tilfellet er modellen *ganske bra*.

I fremtidige leksjoner om klassifiseringer vil du lære hvordan du kan forbedre modellens resultater (for eksempel ved å håndtere ubalanserte data i dette tilfellet).

## 🚀Utfordring

Det er mye mer å utforske når det gjelder logistisk regresjon! Men den beste måten å lære på er å eksperimentere. Finn et datasett som egner seg for denne typen analyse og bygg en modell med det. Hva lærer du? Tips: prøv [Kaggle](https://www.kaggle.com/search?q=logistic+regression+datasets) for interessante datasett.

## Gjennomgang og Selvstudium

Les de første sidene av [denne artikkelen fra Stanford](https://web.stanford.edu/~jurafsky/slp3/5.pdf) om noen praktiske bruksområder for logistisk regresjon. Tenk på oppgaver som egner seg bedre for den ene eller den andre typen regresjonsoppgaver vi har studert så langt. Hva ville fungert best?



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi tilstreber nøyaktighet, vennligst vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for eventuelle misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
